In [1]:
!pip install tensorflow_addons

     |████████████████████████████████| 1.1 MB 5.4 MB/s 


In [2]:
!pip install SimpleITK

     |████████████████████████████████| 48.4 MB 2.0 MB/s 


In [3]:
!pip install dltk

     |████████████████████████████████| 294 kB 5.3 MB/s 
  Created wheel for dltk: filename=dltk-0.2.1-py2.py3-none-any.whl size=37273 sha256=e31a3c2b39a6606a8255e43073beed88df030988abd57840921bc1f573b4d9aa
  Stored in directory: /root/.cache/pip/wheels/e1/61/ca/68587393ae804a47ea2f92cdcfb8e22e704cded914cb07dd4b
Successfully built dltk


In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
%cd drive/MyDrive/The\ Final\ Problem

/content/drive/MyDrive/The Final Problem


In [3]:
import SimpleITK as sitk
import tensorflow as tf
import os
import numpy as np
import random
import _pickle as pickle
from dltk.io.augmentation import add_gaussian_noise, flip, extract_class_balanced_example_array
from dltk.io.preprocessing import whitening
import matplotlib.pyplot as plt
from unet_3d import unet3d
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint, ReduceLROnPlateau
from tensorflow.keras.metrics import FalsePositives, FalseNegatives, Precision, Recall, TruePositives, TrueNegatives
from metrics import Semantic_loss_functions
from tensorflow.keras.optimizers import Adam
from utility import *

In [4]:
train_directory = os.path.join('dataset1','train')
val_directory = os.path.join('dataset1','val')
test_directory = os.path.join('dataset1','test')

shape = (1,32,256,256)
s = Semantic_loss_functions()

semantic loss functions initialized


In [5]:
model = unet3d(shape)

In [6]:
metrics = [s.generalized_dice_coefficient,FalsePositives(), FalseNegatives(),TruePositives(),TrueNegatives()]

In [7]:
print(model.summary())

Model: "model"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_1 (InputLayer)           [(None, 1, 32, 256,  0           []                               
                                 256)]                                                            
                                                                                                  
 conv3d (Conv3D)                (None, 16, 32, 256,  448         ['input_1[0][0]']                
                                 256)                                                             
                                                                                                  
 batch_normalization (BatchNorm  (None, 16, 32, 256,  64         ['conv3d[0][0]']                 
 alization)                      256)                                                         

In [8]:
model.compile(
    optimizer=Adam(lr=0.0001),
    loss=s.tversky_loss,
    metrics=metrics)

/usr/local/lib/python3.7/dist-packages/keras/optimizer_v2/adam.py:105: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  super(Adam, self).__init__(name, **kwargs)


In [9]:
# model.summary()

In [10]:

checkpoint = ModelCheckpoint("best_model.hdf5", monitor='loss', verbose=1,
    save_best_only=True, mode='auto', save_freq='epoch')

model_EarlyStopping = EarlyStopping(monitor='val_loss',
                                    patience=3,
                                    restore_best_weights=True)
lr_decay = ReduceLROnPlateau(
    monitor="val_loss",
    factor=0.1,
    patience=1,
    verbose=0,
    mode="auto",
    min_delta=0.001,
    cooldown=0,
    min_lr=0,
)

In [ ]:
model.fit(
    x=train_generate(train_directory),
    steps_per_epoch=1072,
    validation_data=val_generate(val_directory),
    validation_steps = 368,
    epochs=15,
    callbacks=[model_EarlyStopping, checkpoint, lr_decay]
)

Epoch 1/15
 758/1072 [====================>.........] - ETA: 36:41 - loss: 0.9943 - generalized_dice_coefficient: 0.0012 - false_positives: 126767616.0000 - false_negatives: 2866.0000 - true_positives: 202485.0000 - true_negatives: 1462667904.0000

In [ ]:
model.save('model.h5')

In [ ]:
model = tf.keras.models.load_model('best_model.hdf5',custom_objects={'dice_coefficient_loss': dice_coefficient_loss, 'dice_coefficient':dice_coefficient})

In [ ]:
sample_folder = os.path.join(test_directory,'10054B')
image,mask = load_test_sample(sample_folder)
image.shape, mask.shape

In [ ]:
img_patches,mask_patches,sample_locs = sliding_window(image,mask,shape[1:])

img_patches.shape,mask_patches.shape,sample_locs.shape

In [ ]:
model.evaluate(img_patches, mask_patches, batch_size=1)

In [ ]:
predicitons = model.predict(img_patches,batch_size=1)

In [ ]:
np.array(predicitons).shape

In [ ]:
np.array(predicitons).sum()

In [ ]:
mask_patches.sum()

In [ ]:
TRAIN EPOCH ON 3 CLASSES